# Building an image retrieval system with deep features


# Fire up GraphLab Create
(See [Getting Started with SFrames](../Week%201/Getting%20Started%20with%20SFrames.ipynb) for setup instructions)

In [1]:
import graphlab

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
/opt/conda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
# Limit number of worker processes. This preserves system memory, which prevents hosted notebooks from crashing.
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

/opt/conda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:266: SubjectAltNameWarning: Certificate for beta.graphlab.com has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SubjectAltNameWarning
[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1519132432.log


This non-commercial license of GraphLab Create for academic use is assigned to mmaghraby134@gmail.com and will expire on November 15, 2018.


# Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [64]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

# Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [4]:
# deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
# image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [5]:
image_train.head()

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 10

Data:
+-----+----------------------+------------+-------------------------------+
|  id |        image         |   label    |         deep_features         |
+-----+----------------------+------------+-------------------------------+
|  24 | Height: 32 Width: 32 |    bird    | [0.242871761322, 1.0954537... |
|  33 | Height: 32 Width: 32 |    cat     | [0.525087952614, 0.0, 0.0,... |
|  36 | Height: 32 Width: 32 |    cat     | [0.566015958786, 0.0, 0.0,... |
|  70 | Height: 32 Width: 32 |    dog     | [1.12979578972, 0.0, 0.0, ... |
|  90 | Height: 32 Width: 32 |    bird    | [1.71786928177, 0.0, 0.0, ... |
|  97 | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
| 107 | Height: 32 Width: 32 |    dog     | [0.0, 0.0, 0.220677852631,... |
| 121 | Height: 32 Width: 32 |    bird    | [0.0, 0.23753464222, 0.0, ... |
| 136 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 138 | Height: 32 Width: 32 |    bird    | [0.658935725689, 0.0, 0.0,... |
+-----+----------------------+------------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [73.0, 77.0, 58.0, 71.0, 6... |
| [7.0, 5.0, 8.0, 7.0, 5.0, ... |
| [169.0, 122.0, 65.0, 131.0... |
| [154.0, 179.0, 152.0, 159.... |
| [216.0, 195.0, 180.0, 201.... |
| [33.0, 44.0, 27.0, 29.0, 4... |
| [97.0, 51.0, 31.0, 104.0, ... |
| [93.0, 96.0, 88.0, 102.0, ... |
| [35.0, 59.0, 53.0, 36.0, 5... |
| [205.0, 193.0, 195.0, 200.... |
+-------------------------------+
[10 rows x 5 columns]

# Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [6]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

# Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [9]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [10]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 38.315ms     |

| Done         |         | 100         | 388.107ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |       384       |      0.0      |  1   |
|      0      |       6910      | 36.9403137951 |  2   |
|      0      |      39777      | 38.4634888975 |  3   |
|      0      |      36870      | 39.7559623119 |  4   |
|      0      |      41734      | 39.7866014148 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

We are going to create a simple function to view the nearest neighbors to save typing:

In [11]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [12]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 42.416ms     |

| Done         |         | 100         | 386.626ms    |

+--------------+---------+-------------+--------------+

In [13]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

## Finding similar images to a car

In [14]:
car = image_train[8:9]
car['image'].show()

In [15]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 33.201ms     |

| Done         |         | 100         | 376.677ms    |

+--------------+---------+-------------+--------------+

# Just for fun, let's create a lambda to find and show nearest neighbor images

In [29]:
show_neighbors_from_train = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [30]:
show_neighbors_from_train(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 18.209ms     |

| Done         |         | 100         | 337.502ms    |

+--------------+---------+-------------+--------------+

In [31]:
show_neighbors_from_train(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 13.928ms     |

| Done         |         | 100         | 363.779ms    |

+--------------+---------+-------------+--------------+

# summary the train data

In [19]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


# Creating category-specific image retrieval models: 

In [20]:
cat_image = image_train.filter_by('cat','label')

In [ ]:
cat_image[0]

In [21]:
dog_image = image_train.filter_by('dog','label')

In [22]:
bird_image = image_train.filter_by('bird','label')

In [23]:
automobile_image = image_train.filter_by('automobile','label')

In [25]:
cat_image_model = graphlab.nearest_neighbors.create(cat_image, features = ['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

In [26]:
dog_image_model = graphlab.nearest_neighbors.create(dog_image, features = ['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

In [27]:
bird_image_model = graphlab.nearest_neighbors.create(bird_image, features = ['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

In [28]:
automobile_image_model = graphlab.nearest_neighbors.create(automobile_image, features = ['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

# find closest cats for the first image in the test data

In [65]:
def get_images_from_ids_from_specific_models(query_result,data):
    return data.filter_by(query_result['reference_label'],'id')

In [66]:
show_neighbors_from_test_data = lambda i,model,data: get_images_from_ids_from_specific_models(model.query(image_test[i:i+1]),data)['image'].show()

In [67]:
cat_image_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 19.588ms     |

| Done         |         | 100         | 147.472ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [68]:
show_neighbors_from_test_data(0,cat_image_model,cat_image)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 21.678ms     |

| Done         |         | 100         | 134.183ms    |

+--------------+---------+-------------+--------------+

# find closest dogs for the first image in the test data

In [69]:
dog_image_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 11.915ms     |

| Done         |         | 100         | 138.642ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16976      | 37.4642628784 |  1   |
|      0      |      13387      | 37.5666832169 |  2   |
|      0      |      35867      | 37.6047267079 |  3   |
|      0      |      44603      | 37.7065585153 |  4   |
|      0      |       6094      | 38.5113254907 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [70]:
show_neighbors_from_test_data(0,dog_image_model,dog_image)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 19.977ms     |

| Done         |         | 100         | 138.509ms    |

+--------------+---------+-------------+--------------+